<a href="https://colab.research.google.com/github/ShareAndShine/Lemalabs/blob/main/Classification_Supervised_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
path = '/gdrive/My Drive/Online_ML : Aug 2021/data/'

In [37]:
data = pd.read_csv(path+'paint_mnist.csv', header=None); # Header = none as we do not have column header in the excel
data.head() # Also remember first column is ouput

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,255,255,255,255,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
data.shape # To get total data size the above raw_head will not show all data. Now, we have data size 3599 rows  785 columns

(3599, 785)

In [58]:
data.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784
count,3599.000000,3599.000000,3599.0,3599.0,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.0,3599.000000,3599.0,3599.0,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,...,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000,3599.000000
mean,1.499861,0.070853,0.0,0.0,0.070853,0.354265,0.495971,0.850236,0.779383,0.850236,1.133648,1.700472,1.842178,2.054737,2.267296,2.125590,1.983884,1.983884,1.771325,2.125590,2.054737,1.558766,1.275354,1.133648,1.204501,0.566824,0.283412,0.141706,0.0,0.070853,0.0,0.0,0.566824,0.566824,1.558766,1.913031,3.188386,4.676299,5.384829,6.376771,...,4.109475,3.542651,4.322034,3.542651,3.400945,2.904974,2.550709,1.913031,1.913031,0.566824,0.212559,0.141706,0.141706,0.141706,0.495971,0.425118,0.850236,1.275354,1.983884,2.125590,1.771325,1.700472,1.558766,2.834121,2.834121,2.692415,1.842178,2.196443,1.062795,1.133648,1.346207,0.495971,1.204501,1.346207,1.346207,0.495971,0.637677,0.850236,0.566824,0.566824
std,1.118314,4.250590,0.0,0.0,4.250590,9.499324,11.236624,14.701952,14.078009,14.701952,16.966883,20.756892,21.598414,22.800889,23.941162,23.187428,22.407459,22.407459,21.181951,23.187428,22.800889,19.878769,17.991074,16.966883,17.486622,12.010784,8.497636,6.010407,0.0,4.250590,0.0,0.0,12.010784,12.010784,19.878769,22.006769,28.338943,34.218618,36.667542,39.822784,...,32.114091,29.850848,32.920203,29.850848,29.255979,27.065346,25.379195,22.006769,22.006769,12.010784,7.360192,6.010407,6.010407,6.010407,11.236624,10.404541,14.701952,17.991074,22.407459,23.187428,21.181951,20.756892,19.878769,26.736999,26.736999,26.067325,21.598414,23.567415,16.430406,16.966883,18.481490,11.236624,17.486622,18.481490,18.481490,11.236624,12.737587,14.701952,12.010784,12.010784
min,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.500000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [39]:
cols = data.shape[1] # get all total columns size 
x = np.array(data.iloc[:, 1:cols]) # prepare input i.e get all rows , start from 1 until the last column 
y = np.array(data.iloc[:, 0:1]) # prepare ouptut i.e get all rows and column 1 as we have  expected output stored in column 1

In [40]:
x.shape, y.shape

((3599, 784), (3599, 1))

In [41]:
b = np.unique(y)


In [42]:
np.unique(x)

array([  0, 255])

In [43]:
classes = b.size
classes

4

In [44]:
# Separate inputs and Outputs
# Introduce X0 which is biased column with constant 1
# Get theta matrix created

from sklearn.model_selection import train_test_split #use this lib to split records into train and test and use it for model prediction accuracy. This is a general appraoch followed in ML
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25, random_state=2) # test_size = 0.25 is used to take 25% of overall data for testing. random_state will pick random rows for test and train
x_train.shape, x_test.shape, y_train.shape, y_test.shape 


((2699, 784), (900, 784), (2699, 1), (900, 1))

In [45]:
 x = np.array(x_train) #convert into array
 y = np.array(y_train)

In [46]:
# Introduce X0 which is biased column with constant 1

rows = x.shape[0]
x = np.insert(x,0, values = np.ones(rows), axis =1)
x.shape

(2699, 785)

In [47]:
# Get theta matrix created
params = x.shape[1]
all_theta = np.zeros((classes, params)) # 4 rows and 786 columns and load all zeros to beign with. Later we can change theta and predict the output
all_theta.shape

(4, 785)

In [48]:
def sigmoid(z):
  return (1/(1+np.exp(-z)))

In [49]:
#print(sigmoid(20000))

In [50]:
def computeCost(theta,x,y):  #Cost function
  theta = np.matrix(theta)
  x = np.matrix(x)
  y = np.matrix(y)

  first = np.multiply(-y, np.log(sigmoid( x * theta.T)))
  second = np.multiply((1-y),(np.log(sigmoid(x*theta.T))))

  cost = np.sum(first -second)
  cost = cost/(2*len(x))
  return cost

In [51]:
#print(x.shape) #playing with train data
computeCost(all_theta,x,y)

1.3862943611198906

In [52]:
# GRadient Function
learning_rate = 2
def gradient(theta, x,y):
  theta = np.matrix(theta)
  x = np.matrix(x)
  y = np.matrix(y)

  error = sigmoid(x*theta.T)
  grad = ((x.T * error) / len(x))
  grad = grad * learning_rate 
  return grad

In [53]:
gradient(all_theta,x,y)

matrix([[1.        , 1.        , 1.        , 1.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        ...,
        [0.75583549, 0.75583549, 0.75583549, 0.75583549],
        [0.47239718, 0.47239718, 0.47239718, 0.47239718],
        [0.47239718, 0.47239718, 0.47239718, 0.47239718]])

In [54]:
xyz = gradient(all_theta,x,y)
print(xyz.shape)

(785, 4)


In [55]:
from scipy.optimize import minimize
def one_vs_all(x,y,classes, new_theta):
  params = x.shape[1] #get columns count: 785
  rows = x.shape[0] #get rows count 2699
  for i in range(0, classes):
    theta = new_theta[i,:] # create a temp theta variable array for each row and all columns

    y_i = np.array([1 if label == i else 0 for label in y]) #this is our expected output
    y_i = np.reshape(y_i, (rows,1)) #convert array into matrix format


    #using gradient function that we have written
    # refers the cost funtion we created and also gradient function we created. if JAC is false the system uses default gradient fucntion in the lib
     
    fmin = minimize(fun=computeCost, x0=theta, args=(x,y_i), method='TNC',jac=gradient) 
    all_theta[i,:] = fmin.x

    print(fmin.fun)
  return all_theta


In [56]:
new_theta = one_vs_all(x,y,classes,all_theta)

0.34657359027997264
0.34657359027997264
0.34657359027997264
0.34657359027997264


In [59]:
def predict_all(x,theta):
  #convert into matrix
  x = np.matrix(x)
  theta = np.matrix(theta)

  #run the model for each training instance
  h = sigmoid(x*theta.T)

  #return the column index that has max. confidence
  h_argmax  = np.argmax(h, axis=1)

  return h_argmax, h

In [62]:
y_pred, h = predict_all(x, new_theta)
print(y_pred.shape)
print(h.shape)

(2699, 1)
(2699, 4)
